In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

model_path = "clasificador_analisis/clasificador/clasificador_tono/classweights_crossentropy/comparativa/ep_4/modelo_final"


tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


c:\Users\Jaime\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def predict_sentiment_with_neutral(text, model, tokenizer, alpha=1.0, neutral_threshold=0.1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).squeeze()

    neutral_score = 1 - (abs(probs[0] - 0.5) ** alpha + abs(probs[2] - 0.5) ** alpha)

    if neutral_score > (1 - neutral_threshold):
        return 1  
    else:
        return int(torch.argmax(probs))


In [ ]:
df = pd.read_excel("clasificador_analisis/clasificador/clasificador_tono/classweights_crossentropy/mejor_test.xlsx")
df = df.rename(columns={"text": "text", "label": "label_manual_num"})

df["label_predicted"] = df["text"].apply(lambda x: predict_sentiment_with_neutral(x, model, tokenizer))

print("\n📊 MATRIZ DE CONFUSIÓN:")
print(confusion_matrix(df["label_manual_num"], df["label_predicted"]))

print("\n📈 CLASIFICATION REPORT:")
print(classification_report(
    df["label_manual_num"],
    df["label_predicted"],
    target_names=["Negativo", "Neutro", "Positivo"],
    digits=3
))


📊 MATRIZ DE CONFUSIÓN:
[[201  17   6]
 [ 25  62   9]
 [ 11   6 178]]

📈 CLASIFICATION REPORT:
              precision    recall  f1-score   support

    Negativo      0.848     0.897     0.872       224
      Neutro      0.729     0.646     0.685        96
    Positivo      0.922     0.913     0.918       195

    accuracy                          0.856       515
   macro avg      0.833     0.819     0.825       515
weighted avg      0.854     0.856     0.854       515

